# DS-SF-34 | 11 | Logistic Regression | Assignment | Starter Code

In [56]:
import os

import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 20)
pd.set_option('display.notebook_repr_html', True)
pd.set_option('display.max_columns', 20)

from sklearn import linear_model, model_selection

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

## Probability, Odds, and Odds Ratios

**Probability:** The number of ways that an event can occur divided by the total number of possible outcomes.

The probability of drawing a red card from a standard deck of cards is 26/52 (50 percent).
The probability of drawing a club from that deck is 13/52 (25 percent).

> ### Question 1.  What's the probability of getting heads in a fair coin flip?

In [3]:
# TODO
p=1./2
print p

0.5


The odds for an event is the ratio of the number of ways the event can occur compared to the number of ways it does not occur.

For example, using the same events as above, the odds for:
- Drawing a red card from a standard deck of cards is 1:1; and
- Drawing a club from that deck is 1:3.

> ### Question 2.  What's the odds of a fair coin flip?

In [ ]:
# TODO
1:1

> ### Question 3.  Suppose that 18 out of 20 patients in an experiment lost weight while using diet A, while 16 out of 20  lost weight using diet B.  What's the probability of weight loss with diet A?  What's the odds?

In [ ]:
# TODO
probability = 18/20
odds of weight loss with diet A = 18:2  9:1   

> ### Question 4.  What's the probablity of weight loss with diet B?  What are the odds?

In [ ]:
# TODO
probability = 16/20
odds of weight loss with diet B = 16:4   4:1

> ### Question 5.  What's the odds ratio?

In [5]:
# TODO
# odds ratio of diet A vs diet B
odds = 9/4.
print odds

2.25


## Bank Marketing

In [57]:
df = pd.read_csv(os.path.join('..', 'datasets', 'dataset-11-bank-marketing.csv'))
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,c
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no


> The dataset is related to the direct marketing campaigns (by phone) of a Portuguese banking institution.  The classification goal is to predict if the client will subscribe a term deposit (variable y).

Attributes Information:

- Input variables:
  - [Bank client data]
    - `age` (numeric)
    - `job`: type of job (categorical)
    - `marital`: marital status (categorical)
      - Note: `divorced` means divorced or widowed)
    - `education` (categorical)
    - `default`: has credit in default? (categorical)
    - `balance`: bank account balance (\$)
    - `housing`: has housing loan? (categorical)
    - `loan`: has personal loan? (categorical)
  - [Data related with the last contact of the current campaign]
    - `contact`: contact communication type (categorical)
    - `month`: last contact month of year (categorical)
    - `day_of_week`: last contact day of the week (categorical)
    - `duration`: last contact duration, in seconds (numeric)
      - Important note: this attribute highly affects the output target (e.g., if `duration = 0` then `y = 'no'`). Yet, the duration is not known before a call is performed.  Also, after the end of the call y is obviously known.  Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
  - [Other attributes]
    - `campaign`: number of contacts performed during this campaign and for this client (numeric)
    - `pdays`: number of days that passed by after the client was last contacted from a previous campaign (numeric)
      - 999 means client was not previously contacted
    - `previous`: number of contacts performed before this campaign and for this client (numeric)
    - `poutcome`: outcome of the previous marketing campaign (categorical)

- Output variable (desired target):
  - `y`: has the client subscribed a term deposit? (binary)

In [18]:
df.dtypes

age           int64
job          object
marital      object
education    object
default      object
balance       int64
housing      object
loan         object
contact      object
day           int64
month        object
duration      int64
campaign      int64
pdays         int64
previous      int64
poutcome     object
c            object
dtype: object

> Our goal is to develop a model the best predicts the outcome `y`, the success of the marketing campaign.

> ## Question 6.  Remove the categorical variables with the most number of distinct values

In [65]:
# TODO  
   # 12   -- this one has the most number of distinct values
#len(df.job.value_counts(sort=False))
   # 3  married  2797 divorced   528 single  1196
# df.marital.value_counts(sort=False)
   # 4
#len(df.education.value_counts(sort=False))

# 2  yes  76 no 4445  -- turn this into default_no    
#df.default.value_counts(sort=False)

# 2  yes value 2559   no values 1962  -- leave this as housing
# df.housing.value_counts(sort=False)

# 2  yes 691 no  3830  -- turn this into loan_no
# df.loan.value_counts(sort=False)

   # 3   cellular   2896 unknown  1324 telephone  301
# df.contact.value_counts(sort=False)
    # 4   success   129  unknown    3705 failure   490 other 197
df.poutcome.value_counts(sort=False)

# 2   yes 521 no 4000  -- turn this into subscribed_no
#df.c.value_counts(sort=False)


success     129
unknown    3705
failure     490
other       197
Name: poutcome, dtype: int64

In [59]:
df.drop(['job'], axis = 1, inplace = True)


> ## Question 7.  Recode all `yes`/`no` categorical variables with `0` as the most frequent value (then also append `"_no"` to the variable name), and `1` for the other (then leave the name unchanged)

In [61]:
# TODO
  # default
df['default_no'] = (df.default=='no')*1
df.drop(['default'], axis = 1, inplace = True)
   # housing
df['housing'] = (df.housing=='yes')*1
   # loan
df['loan_no'] = (df.loan=='no')*1
df.drop(['loan'], axis = 1, inplace = True)
   # subscribed
df['subscribed_no'] = (df.c=='no')*1
df.drop(['c'], axis = 1, inplace = True)
    

In [62]:
df

,age,marital,education,balance,housing,contact,day,month,duration,campaign,pdays,previous,poutcome,default_no,loan_no,subscribed_no
0,30,married,primary,1787,0,cellular,19,oct,79,1,-1,0,unknown,1,1,1
1,33,married,secondary,4789,1,cellular,11,may,220,1,339,4,failure,1,0,1
2,35,single,tertiary,1350,1,cellular,16,apr,185,1,330,1,failure,1,1,1
3,30,married,tertiary,1476,1,unknown,3,jun,199,4,-1,0,unknown,1,0,1
4,59,married,secondary,0,1,unknown,5,may,226,1,-1,0,unknown,1,1,1
5,35,single,tertiary,747,0,cellular,23,feb,141,2,176,3,failure,1,1,1
6,36,married,tertiary,307,1,cellular,14,may,341,1,330,2,other,1,1,1
7,39,married,secondary,147,1,cellular,6,may,151,2,-1,0,unknown,1,1,1
8,41,married,tertiary,221,1,unknown,14,may,57,2,-1,0,unknown,1,1,1
9,43,married,primary,-88,1,cellular,17,apr,313,1,147,2,failure,1,0,1


> ## Question 8.  Create binary/dummy variables for the other categorical variables

In [ ]:
# TODO
   # 3  married divorced single
# df.marital.value_counts(sort=False)
   # 4
#len(df.education.value_counts(sort=False))

   # 3   cellular unknown telephone
# df.contact.value_counts(sort=False)
    # 4   success  unknown  failure  other
#df.poutcome.value_counts(sort=False)


> ## Question 9.  What should be your baseline for these binary variables (namely, which binary variables should you not include in your model)?

In [ ]:
# TODO

Answer: TODO

> ## Question 10.  What input variable in the dataset seems to predict the outcome quite well.  Why?

In [ ]:
# TODO

Answer: TODO

> ## Question 11.  Split the dataset into a training set (60%) and a testing set (the rest)

In [ ]:
# TODO

> ## Question 12.  Run a logistic regression with `age`, `marital` (the dummies), `default`, `balance`, `housing`, `loan`, `campaign`, `pdays`, `previous`?

In [ ]:
# TODO

> ## Question 13.  What is your training error?  What is your generalization error?  Does it make sense?

In [ ]:
# TODO

Answer: TODO

> ## Question 14.  Interpret your coefficients. (At least `marital_single`, `campaign`, and `default`).  Does your interpretation  make sense?

In [ ]:
# TODO

Answer: TODO

> ## Question 15.  What is your prediction for a 30 years old single female, a homeowner with a \$1,000 balance in the bank, without a loan, who has never been contacted before, and who has never defaulted

In [ ]:
# TODO

Answer: TODO